In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "2"  # Set the GPU 2 to use

## 무비렌즈

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# gpu 설정
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

### 데이터 준비

In [ ]:
# 데이터 로드

"""
필요한 컬럼은 유저, 아이템, rating

우선 간단하게 빨리 해보는게 중요하니,
rating이 5점이면 rating 컬럼을 1
아니라면 0로 바꾸자고.
"""


def getDataByScenario(scenario):
    """
    :param scenario: increase, fixed, user, item
    :return: dfs
    """
    dfs = []

    if scenario in ["increase", 'fixed']:
        for i in range(6):
            df = pd.read_csv(f"./dataset/Movielens/{scenario}/ml_1m_inc{i}.csv")
            dfs.append(df)

    if scenario in ["user", "item"]:
        for i in range(6):
            train = pd.read_csv(f"./dataset/Movielens/{scenario}/train_ml_1m_inc{i}.csv")
            test = pd.read_csv(f"./dataset/Movielens/{scenario}/test_ml_1m_inc{i}.csv")
            dfs.append((train, test))

    return dfs

## dataloader 정의

In [ ]:
class MovielensDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx]['user']
        item = self.df.iloc[idx]['item']
        rating = self.df.iloc[idx]['rating']
        return user, item, rating

## 모델 정의

In [ ]:
# NCF 모델
class NCF(nn.Module):
    def __init__(self, n_users, n_movies, emb_size=8, hidden_size=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(n_users, emb_size)
        self.movie_embedding = nn.Embedding(n_movies, emb_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, user_input, movie_input):
        user_embedded = self.user_embedding(user_input)
        movie_embedded = self.movie_embedding(movie_input)
        input_concat = torch.cat([user_embedded, movie_embedded], dim=-1)
        prediction = self.fc_layers(input_concat)
        return prediction

## 모델 train/test 함수 정의

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [ ]:
def recall_at_k(output, target, k):
    if len(output) < k:
        k = len(output)
    _, idx = torch.topk(output, k=k)
    hit = torch.sum(target[idx])
    return hit.float() / target.sum().float() if target.sum().float() else torch.Tensor([0])


def test(model, device, test_loader, k=20):
    model.eval()
    criterion = nn.BCELoss()

    test_loss = 0
    test_recall = 0
    with torch.no_grad():
        for user, item, rating in test_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            output = model(user, item).squeeze()
            loss = criterion(output, rating.float())
            test_loss += loss.item()
            test_recall += recall_at_k(output, rating, k).item()  # recall@20 기준
    test_loss /= len(test_loader)
    test_recall /= len(test_loader)

    return test_loss, test_recall

### 모델 학습

0. Full
1. Naive
2. EWC

In [ ]:
### Config..
EPOCH = 1
SEED = 42
BATCH_SIZE = 64
N_USER = 6040
N_ITEM = 3952

0. Full

In [ ]:
def concat_df(dfs, k):
    return pd.concat(dfs[:k+1], axis=0)

In [ ]:
def getFullResultByScenario(scenario):
    recall_list = []
    dfs = getDataByScenario(scenario)

    for i, df in enumerate(dfs):
        # 모델 객체 생성
        n_users = N_USER + 1
        n_movies = N_ITEM + 1
        model = NCF(n_users, n_movies).to(device)
        # 옵티마이저 설정
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        if i == 0:
            # base block train-test
            if scenario in ["increase", "fixed"]:
                train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=SEED)
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

        else:
            # inc block train-test
            if scenario in ["increase", "fixed"]:
                #
                if i == len(dfs)-1:
                    break
                # train dataset은 0~i를 모두 concat한 것
                train_dataset = concat_df(dfs, i)
                test_dataset = dfs[i+1]

            elif scenario in ["user", "item"]:
                temp_dfs =  []
                for j in range(i):
                    temp_dfs.append(dfs[j][0])
                train_dataset = concat_df(temp_dfs, i)
                _, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

            # test
            """
            user or item 시나리오의 경우,
            현재 모델에 대해서,
            이전 test 데이터들의 recall@20,
            현재 test 데이터에 대한 recall@20
            그리고 그 값에 대한 평균을 구해야 한다.
            """
            if scenario in ["user", "item"]:
                recall20_prev = []
                for j in range(i+1):
                    _, test_dataset = dfs[j]
                    test_dataset = MovielensDataset(test_dataset)
                    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
                    _, recall20 = test(model, device, test_loader)
                    recall20_prev.append(recall20)
                    print(f"----- {scenario} scenario, when Task is {i}, prev Task {j} recall 20 = {recall20}")
                avg_prev_recall = sum(recall20_prev) / len(recall20_prev)
                print(f"{scenario} scenario avg prev recall : {avg_prev_recall}")

    avg_recall = sum(recall_list) / len(recall_list)
    print(f"{scenario} scenario avg recall : {avg_recall}")
    if scenario in ["user", "item"]:
        return  (avg_recall, avg_prev_recall)
    return avg_recall

In [ ]:
fullIncrease = getFullResultByScenario("increase")

In [ ]:
fullFixed = getFullResultByScenario("fixed")

In [ ]:
fullUser1, fullUser2 = getFullResultByScenario("user")

In [ ]:
fullItem1, fullItem2 = getFullResultByScenario("item")

1. Naive

우선 모든 데이터에 대해 incremental training을 하고 test해보자

In [ ]:
def getNaiveResultByScenario(scenario):
    recall_list = []
    dfs = getDataByScenario(scenario)
    # 모델 객체 생성
    n_users = N_USER + 1
    n_movies = N_ITEM + 1
    model = NCF(n_users, n_movies).to(device)
    # 옵티마이저 설정
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for i, df in enumerate(dfs):

        if i == 0:
            # base block train-test

            if scenario in ["increase", "fixed"]:
                train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=SEED)
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

        else:
            # inc block train-test
            # 모든 시나리오 마지막은 생략한다..
            # if i == len(dfs)-1:
            #         break

            # 데이터 준비
            if scenario in ["increase", "fixed"]:
                #
                if i == len(dfs)-1:
                    break
                train_dataset = df
                test_dataset = dfs[i+1]
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train(model, device, train_loader, optimizer, e)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

            # test
            """
            user or item 시나리오의 경우,
            현재 모델에 대해서,
            이전 test 데이터들의 recall@20,
            현재 test 데이터에 대한 recall@20
            그리고 그 값에 대한 평균을 구해야 한다.
            """
            if scenario in ["user", "item"]:
                recall20_prev = []
                for j in range(i+1):
                    _, test_dataset = dfs[j]
                    test_dataset = MovielensDataset(test_dataset)
                    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
                    _, recall20 = test(model, device, test_loader)
                    recall20_prev.append(recall20)
                    print(f"----- {scenario} scenario, when Task is {i}, prev Task {j} recall 20 = {recall20}")
                avg_prev_recall = sum(recall20_prev) / len(recall20_prev)
                print(f"{scenario} scenario avg prev recall : {avg_prev_recall}")

    avg_recall = sum(recall_list) / len(recall_list)
    print(f"{scenario} scenario avg recall : {avg_recall}")
    if scenario in ["user", "item"]:
        return  (avg_recall, avg_prev_recall)
    return avg_recall

In [ ]:
naiveIncrease = getNaiveResultByScenario("increase")

In [ ]:
naivefixed = getNaiveResultByScenario("fixed")

In [ ]:
naiveUser1, naiveUser2 = getNaiveResultByScenario("user")

In [ ]:
naiveItem1, naiveItem2 = getNaiveResultByScenario("item")

2. EWC

In [ ]:
# Task가 끝날 때 마다 optpar와 fisher를 저장해주는 함수.
def on_task_update(model, device, train_loader, optimizer, task_id, fisher_dict, optpar_dict):
    model.train()
    criterion = nn.BCELoss()
    optimizer.zero_grad()

    # accumulating gradients
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()

    fisher_dict[task_id] = {}
    optpar_dict[task_id] = {}

    # gradients accumulated can be used to calculate fisher
    for name, param in model.named_parameters():
        fisher_dict[task_id][name] = param.grad.data.clone().pow(2)  # 누적 grad 값
        optpar_dict[task_id][name] = param.data.clone()  # 최적 grad 값

In [ ]:
# EWC를 적용한 train 함수
def train_ewc(model, device, train_loader, optimizer, epoch, task_id, fisher_dict, optpar_dict, ewc_lambda):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        train_loss += loss.item()

        # EWC 적용 부분
        for task in range(task_id):
            for name, param in model.named_parameters():
                fisher = fisher_dict[task][name]
                optpar = optpar_dict[task][name]
                train_loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda

        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [ ]:
def getEWCResultByScenario(scenario):
    recall_list = []
    dfs = getDataByScenario(scenario)

    # 모델 객체 생성
    n_users = N_USER + 1
    n_movies = N_ITEM + 1
    model = NCF(n_users, n_movies).to(device)
    # 옵티마이저 설정
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # EWC에 필요한 변수
    fisher_dict = {}
    optpar_dict = {}
    ewc_lambda = 0.4  # ewc 강도 조절.. 높을수록 이전 파라미터의 중요도가 높아짐

    for i, df in enumerate(dfs):
        if i == 0:
            # base block train-test

            if scenario in ["increase", "fixed"]:
                train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=SEED)
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train_ewc(model, device, train_loader, optimizer, e, i, fisher_dict, optpar_dict, ewc_lambda)
            on_task_update(model, device, train_loader, optimizer, i, fisher_dict, optpar_dict)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

        else:
            # inc block train-test

            # 마지막은 생략한다..
            # if i == len(dfs)-1:
            #         break

            # 데이터 준비
            if scenario in ["increase", "fixed"]:
                if i == len(dfs)-1:
                    break
                train_dataset = df
                test_dataset = dfs[i+1]
            elif scenario in ["user", "item"]:
                train_dataset, test_dataset = df

            train_dataset = MovielensDataset(train_dataset)
            test_dataset = MovielensDataset(test_dataset)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

            # train
            epoch = EPOCH
            # print(f"************** Train Start At TASK{i}")
            for e in tqdm(range(1, epoch + 1)):
                train_ewc(model, device, train_loader, optimizer, e, i, fisher_dict, optpar_dict, ewc_lambda)
            on_task_update(model, device, train_loader, optimizer, i, fisher_dict, optpar_dict)

            # test
            _, recall20 = test(model, device, test_loader)
            recall_list.append(recall20)
            print(f"******* {scenario} scenario At {i} TASK recall20 = {recall20}")

            # test
            """
            user or item 시나리오의 경우,
            현재 모델에 대해서,
            이전 test 데이터들의 recall@20,
            현재 test 데이터에 대한 recall@20
            그리고 그 값에 대한 평균을 구해야 한다.
            """
            if scenario in ["user", "item"]:
                recall20_prev = []
                for j in range(i+1):
                    _, test_dataset = dfs[j]
                    test_dataset = MovielensDataset(test_dataset)
                    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
                    _, recall20 = test(model, device, test_loader)
                    recall20_prev.append(recall20)
                    print(f"----- {scenario} scenario, when Task is {i}, prev Task {j} recall 20 = {recall20}")
                avg_prev_recall = sum(recall20_prev) / len(recall20_prev)
                print(f"{scenario} scenario avg prev recall : {avg_prev_recall}")

    avg_recall = sum(recall_list) / len(recall_list)
    print(f"{scenario} scenario avg recall : {avg_recall}")
    if scenario in ["user", "item"]:
        return  (avg_recall, avg_prev_recall)
    return avg_recall

In [ ]:
ewcIncrease = getEWCResultByScenario("increase")

In [ ]:
ewcfixed = getEWCResultByScenario("fixed")

In [ ]:
ewcUser1, ewcUser2 = getEWCResultByScenario("user")

In [ ]:
ewcItem1, ewcItem2 = getEWCResultByScenario("item")

In [ ]:
print(f"""
naiveIncrease: {naiveIncrease}
naivefixed: {naivefixed}
naiveUser1: {naiveUser1}
naiveUser2: {naiveUser2}
naiveItem1" {naiveItem1}
naiveItem2" {naiveItem2}
""")

In [ ]:
print(f"""
ewcIncrease: {ewcIncrease}
ewcfixed: {ewcfixed}
ewcUser1: {ewcUser1}
ewcUser2: {ewcUser2}
ewcItem1" {ewcItem1}
ewcItem2" {ewcItem2}
""")

In [ ]:
print(f"""
{ewcIncrease-naiveIncrease}
{ewcfixed-naivefixed}
{ewcUser1-naiveUser1}
{ewcUser2-naiveUser2}
{ewcItem1-naiveItem1}
{ewcItem2-naiveItem2}
""")